In [0]:
%run "../includes/configuration"

In [0]:
movie_df = spark.read.parquet(f"{silver_folder_path}/movies").filter("year_release_date >= 2010")

In [0]:
production_country_df = spark.read.parquet(f"{silver_folder_path}/productions_countries")

In [0]:
country_df = spark.read.parquet(f"{silver_folder_path}/countries")

In [0]:
prod_company_df = spark.read.parquet(f"{silver_folder_path}/productions_companies")

In [0]:
movie_company_df = spark.read.parquet(f"{silver_folder_path}/movies_companies")

## Join Country y production_country

In [0]:
country_prod_ctry_df = country_df.join(production_country_df, 
              country_df.country_id == production_country_df.country_id, 'inner').select(
                  country_df.country_name, production_country_df.movie_id
              )

## Join production_company y movie_company

In [0]:
prod_company_mov_comp = prod_company_df.join(movie_company_df, 
              prod_company_df.company_id == movie_company_df.company_id, 'inner').select(
                  movie_company_df.movie_id, prod_company_df.company_name)

## Join movie_df, country_prod_ctry, prod_company_mov_comp

In [0]:
results_df = movie_df.join(country_prod_ctry_df, movie_df.movie_id == country_prod_ctry_df.movie_id, 'inner') \
            .join(prod_company_mov_comp, movie_df.movie_id == prod_company_mov_comp.movie_id, 'inner') \
            .select(movie_df.title, movie_df.budget, movie_df.revenue, movie_df.duration_time,
                    movie_df.release_date, country_prod_ctry_df.country_name, prod_company_mov_comp.company_name)

- Agregar columna "created_date"

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
results_df = results_df.withColumn("created_date", current_timestamp()).orderBy("title")

In [0]:
display(results_df)

## Escribir en formato parquet en el DataLake

In [0]:
# results_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_country_prod_company")
results_df.write.mode("overwrite").format('parquet').saveAsTable("movie_gold.results_country_prod_company")

In [0]:
display(spark.read.parquet(f"{gold_folder_path}/results_country_prod_company"))